# An attempt to identify the electronic ghosts found in SM

In [1]:
import numpy as np
# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib
matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

In [2]:
from extraction_sm import *
from astropy.io import fits

In [89]:
# pick out all files for one fov and row

fov = 2
row = 5
pathfile = "split_filenames/filenames_sm_{}_row_{}.txt".format(fov,row)


allfiles = []
f = open(pathfile, 'r')
allfiles = f.readlines()
f.close()

# strip away newlines and add absolute path prefix
allfiles = ["../../"+s.strip() for s in allfiles]

sourcefiles = allfiles[:]

print("Using", len(sourcefiles), "files")

Using 1185 files


In [90]:
# write a function that collects the ghosts

def sm_starmask_ghosts(image,threshold):
    import numpy as np
    from scipy.ndimage import generate_binary_structure
    from scipy.ndimage.morphology import binary_dilation
    from scipy.ndimage.measurements import label, find_objects

    # occasionally, an overflow for very bright stars causes some pixels to be set to 0
    # -> set them to 65535, so they will be masked
    image[image==0] = 65535

    # number of ghosts detected
    outnum = 0
    
    # initialize the mask to all False
    ######starmask = np.zeros(image.shape, dtype=bool)

    # if nothing is saturated, return the starmask
    if np.max(image)<65535:
        #return starmask
        return outnum
    else:
        # mask of pixels above threshold
        satmap = np.logical_and(image,image>=threshold)

        # dilate it - sometimes, pixel values fluctuate around the threshold
        satmap = binary_dilation(satmap,iterations=1)

        # labelling all connected pixels
        (starlabels, nstars) = label(satmap,structure=(np.ones((3,3))))

        # we're only interested in the labels of pixels that are saturated
        # so, get all the labels that have that:
        satlabels = np.unique(starlabels[image==65535])

        for lab in satlabels:
            starcoords = np.argwhere(starlabels==lab)
            ######starmask[starlabels==lab] = 1
            # this star may also cause electronic ghosts at lower AC = x
            # they will have the same central AL, though
            # we can determine that: it's the AL-coordinate of the masked star pixel LOWEST in AC
            edgepix = starcoords[np.where(starcoords[:,1] == np.min(starcoords[:,1]))][0] # first should be accurate enough
            ghostlabels = np.unique(starlabels[edgepix[0],0:edgepix[1]])
            if len(ghostlabels) == 1:
                # Only found 0's, so nothing
                return outnum
                continue
            else:
                for glab in ghostlabels[1:]:
                    if np.sum(starlabels[starlabels==glab]/glab) >= 100:
                        ######starmask[starlabels==glab] = 1
                        outnum += 1

    return outnum
    #return starmask - dilate it once (we won't be losing much here)
    #starmask = binary_dilation(starmask,iterations=1)

In [91]:
# let's test the execution time
import time
timingstart = time.time()

ghostfiles = []

for sourcefile in sourcefiles:
    image = fits.getdata(sourcefile)[:,7:]
    nghosts = sm_starmask_ghosts(image,2000)
    if nghosts > 0:
        ghostfiles.append([sourcefile[24:],nghosts])

    
timingend = time.time()
print("Elapsed time:", timingend - timingstart)

Elapsed time: 361.3627359867096


In [92]:
print(len(ghostfiles))

8


In [93]:
for file in ghostfiles:
    print(file[0], file[1])

2014-05-27/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_18165643999804700_CDP_NONE.fits 1
2015-04-03/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_45102899999131100_CDP_NONE.fits 1
2016-05-15/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_80337467997943100_CDP_NONE.fits 1
2016-05-18/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_80588448000525500_CDP_NONE.fits 1
2016-09-26/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_91884270000023900_CDP_NONE.fits 1
2017-02-04/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_103225758982847900_CDP_NONE.fits 1
2017-03-10/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_106173052000171100_CDP_NONE.fits 1
2017-03-20/SIF_PROCESSING_CCD_ROW_5_SM2_OBMT_START_107008477999142300_CDP_NONE.fits 1
